# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,brookings,44.3114,-96.7984,30.19,57,6,2.20,US,1690149196
1,1,ushuaia,-54.8000,-68.3000,0.81,75,75,4.12,AR,1690149083
2,2,leh,34.1667,77.5833,16.56,58,45,1.78,IN,1690149198
3,3,wesley,15.5667,-61.3167,28.74,79,40,4.63,DM,1690149198
4,4,polyarnyy,69.1989,33.4478,9.06,91,100,6.18,RU,1690149198


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    scale = 1,  
)

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,brookings,44.3114,-96.7984,30.19,57,6,2.20,US,1690149196
1,1,ushuaia,-54.8000,-68.3000,0.81,75,75,4.12,AR,1690149083
2,2,leh,34.1667,77.5833,16.56,58,45,1.78,IN,1690149198
3,3,wesley,15.5667,-61.3167,28.74,79,40,4.63,DM,1690149198
4,4,polyarnyy,69.1989,33.4478,9.06,91,100,6.18,RU,1690149198


In [5]:
# Narrow down cities that fit criteria and drop any results with null values
my_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= 28) & (city_data_df["Max Temp"] >= 20) & (city_data_df["Lat"] > 20) & (city_data_df["Humidity"] < 60)]
# Drop any rows with null values
my_city_data_df = my_city_data_df.dropna()

# Display sample data
my_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,jamestown,42.0970,-79.2353,26.92,55,75,1.79,US,1690149206
41,41,college,64.8569,-147.8028,27.82,49,75,0.89,US,1690149212
42,42,yellowknife,62.4560,-114.3525,22.29,40,100,4.63,CA,1690149212
52,52,peace river,56.2501,-117.2860,23.17,46,100,3.09,CA,1690149216
68,68,guerrero negro,27.9769,-114.0611,26.95,49,3,6.26,MX,1690149221


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_city_data_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
23,jamestown,US,42.0970,-79.2353,55,
41,college,US,64.8569,-147.8028,49,
42,yellowknife,CA,62.4560,-114.3525,40,
52,peace river,CA,56.2501,-117.2860,46,
68,guerrero negro,MX,27.9769,-114.0611,49,
118,thunder bay,CA,48.4001,-89.3168,57,
121,badger,US,64.8000,-147.5333,42,
168,thompson,CA,55.7435,-97.8558,43,
183,dalandzadgad,MN,43.5708,104.4250,56,
184,fort st. john,CA,56.2499,-120.8529,28,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    
    params = {
    "filters": filters,
    "bias": bias,
    "apiKey": geoapify_key
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jamestown - nearest hotel: No hotel found
college - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
peace river - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
thunder bay - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
thompson - nearest hotel: No hotel found
dalandzadgad - nearest hotel: No hotel found
fort st. john - nearest hotel: No hotel found
zhumysker - nearest hotel: No hotel found
chibougamau - nearest hotel: No hotel found
chahar burj - nearest hotel: No hotel found
stettler - nearest hotel: No hotel found
shahin dezh - nearest hotel: No hotel found
dzhangala - nearest hotel: No hotel found
aibak - nearest hotel: No hotel found
namtsy - nearest hotel: No hotel found
xinzhou - nearest hotel: No hotel found
xiva - nearest hotel: No hotel found
borogontsy - nearest hotel: No hotel found
dryden - nearest hotel: No hotel found
zharkent - nearest hotel: No hotel found
tuytepa - ne

,City,Country,Lat,Lng,Humidity,Hotel Name
23,jamestown,US,42.0970,-79.2353,55,No hotel found
41,college,US,64.8569,-147.8028,49,No hotel found
42,yellowknife,CA,62.4560,-114.3525,40,No hotel found
52,peace river,CA,56.2501,-117.2860,46,No hotel found
68,guerrero negro,MX,27.9769,-114.0611,49,No hotel found
118,thunder bay,CA,48.4001,-89.3168,57,No hotel found
121,badger,US,64.8000,-147.5333,42,No hotel found
168,thompson,CA,55.7435,-97.8558,43,No hotel found
183,dalandzadgad,MN,43.5708,104.4250,56,No hotel found
184,fort st. john,CA,56.2499,-120.8529,28,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE